In [ ]:
auth_key = ''

In [2]:
import urllib3

urllib3.disable_warnings()

In [ ]:
from gigachat import GigaChat

# Инициализируем клиент с отключенным SSL-верификацией
giga = GigaChat(credentials=auth_key, verify_ssl_certs=False)

# Получаем токен
response = giga.get_token()

access_token = response.access_token

In [ ]:
import requests
import json
from datetime import datetime

# Функция, которую будем отправлять
def get_current_time():
    return datetime.now().isoformat()

# URL и токен доступа
url = "https://gigachat.devices.sberbank.ru/api/v1/chat/completions"
auth_header = {"Authorization": f"Bearer {access_token}"}

functions = [
    {
        "name": "get_current_time",
        "description": "Возвращает текущее время в ISO формате",
        "parameters": {
            "type": "object",
            "properties": {}  # важно оставить пустой объект properties
        },
        "return_parameters": {
            "type": "object",
            "properties": {
                "time": {
                    "type": "string",
                    "description": "Текущее время в формате ISO"
                }
            },
            "required": ["time"]
        }
    }
]

# Основной запрос
body = {
    "model": "GigaChat",
    "messages": [
        {"role": "user", "content": "Какое сейчас время?"}
    ],
    "functions": functions,
    "function_call": "auto"
}

# Отправляем запрос
response = requests.post(url, headers={"Content-Type": "application/json", **auth_header}, data=json.dumps(body), verify=False)

# Анализируем ответ
result = response.json()
print(result)

if "choices" in result and len(result["choices"]) > 0:
    choice = result["choices"][0]
    
    if "function_call" in choice['message']:
        func_call = choice['message']["function_call"]
        
        # Вызываем функцию
        output = locals()[func_call["name"]]()
        print(output)
    else:
        print(choice["message"]["content"])
else:
    print("Ошибка:", result)

# Дополняем ответ
body['messages'].append(choice['message'])
body['messages'].append({'role': 'function', 'content': f"\"{output}\""})

# Отправляем запрос
response = requests.post(url, headers={"Content-Type": "application/json", **auth_header}, data=json.dumps(body), verify=False)

# Анализируем ответ
result = response.json()
print(result['choices'][0]['message']['content'])

{'choices': [{'message': {'content': '', 'role': 'assistant', 'function_call': {'name': 'get_current_time', 'arguments': {}}, 'functions_state_id': '0199a5ac-3c69-73c1-9f53-555c01fcc0d9'}, 'index': 0, 'finish_reason': 'function_call'}], 'created': 1759421152, 'model': 'GigaChat:2.0.28.2', 'object': 'chat.completion', 'usage': {'prompt_tokens': 72, 'completion_tokens': 13, 'total_tokens': 85, 'precached_prompt_tokens': 0}}
2025-10-02T19:05:52.426895
Сейчас 19:05, вторник, 2 октября 2025 года.
